In [3]:
import cv2 as cv
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path="./fashion_mnist_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details: ", input_details)
print("Output details: ", output_details)

class_names = [
  'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
  'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

Input details:  [{'name': 'serving_default_keras_tensor_15:0', 'index': 0, 'shape': array([ 1, 28, 28]), 'shape_signature': array([-1, 28, 28]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details:  [{'name': 'StatefulPartitionedCall_1:0', 'index': 12, 'shape': array([ 1, 10]), 'shape_signature': array([-1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [4]:
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    resized = cv.resize(gray,(28,28))
    img_normalized = resized.astype("float32")/255.0
    
    input_data = np.expand_dims(img_normalized,axis=0)
    
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    
    output_data = interpreter.get_tensor(output_details[0]['index'])
    pred_class = np.argmax(output_data[0])
    conf = np.max(output_data[0])
    
    text = f"{class_names[pred_class]} {conf*100:0.1f}%"
    cv.putText(frame, text, (10,30), 
        cv.FONT_HERSHEY_SIMPLEX,
        1, (0,255,0), 2, cv.LINE_AA            
    )  
    
    cv.imshow("Fashion_mnist_live", frame)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows() 